In [1]:
%load_ext watermark
%watermark

Last updated: 2024-04-18T10:00:29.682714-05:00

Python implementation: CPython
Python version       : 3.10.11
IPython version      : 8.13.2

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 10, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [2]:
from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing, feature_extraction
from sklearn import preprocessing, feature_extraction
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion
#import category_encoders
from sklearn.model_selection import cross_validate
from category_encoders import OneHotEncoder
from category_encoders.ordinal import OrdinalEncoder
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [3]:
consumidores = pd.read_csv("C:/Users/Danny/Desktop/Analisis tesis/4 - Pipelines/data/df_consumo_varObjetivo.csv")

In [4]:
consumidores.head()

,departamento,residentes_hogar,d2_04_num_hijos,per_edad_tipo,estrato_tipo,situacion_tipo,frecuencia_consumo_marihuana_tipo,frecuencia_consumo_cocaina_tipo,frecuencia_consumo_basuco_tipo,per_sexo_tipo,...,g_06_a_posibilidad_conseguir_marihuana_tipo,g_06_b_posibilidad_conseguir_cocaina_tipo,g_06_c_posibilidad_conseguir_basuco_tipo,g_07_alguien_ofrecio_comprar_probar_sp_tipo,g_01_a_num_familiares_consumen_sp_imp_tipo,g_02_a_num_amigos_consumen_sp_imp_tipo,g_08_a_ofrecieron_marihuana_imp_tipo,g_08_b_ofrecieron_cocaina_imp_tipo,g_08_c_ofrecieron_basuco_imp_tipo,CatRiesgo
0,Guaviare,5,0,Joven,Estrato bajo,Trabajando,Nunca,Nunca,Nunca,Hombre,...,g_06_a_No_conoce_facilidad_dificultad,g_06_b_No_conoce_facilidad_dificultad,g_06_c_No_conoce_facilidad_dificultad,g_07_No,g_01_a_No_aplica,g_02_a_Dos_o_mas,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
1,Cordoba,4,3,Adulto,Estrato bajo,Trabajando,Nunca,Nunca,Nunca,Mujer,...,g_06_a_Seria_facil,g_06_b_Seria_facil,g_06_c_Seria_facil,g_07_No,g_01_a_No_aplica,g_02_a_No_aplica,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
2,Caqueta,4,2,Adulto,Estrato bajo,Trabajando,Nunca,Nunca,Nunca,Hombre,...,g_06_a_No_conoce_facilidad_dificultad,g_06_b_No_conoce_facilidad_dificultad,g_06_c_No_conoce_facilidad_dificultad,g_07_No,g_01_a_No_aplica,g_02_a_No_aplica,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
3,Bolivar,6,0,Adulto,Estrato alto,Trabajando,Nunca,Nunca,Nunca,Mujer,...,g_06_a_No_podria_conseguir,g_06_b_No_podria_conseguir,g_06_c_No_podria_conseguir,g_07_No,g_01_a_No_aplica,g_02_a_No_aplica,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
4,Tolima,4,1,Joven,Estrato bajo,Buscando trabajo,Nunca,Nunca,Nunca,Mujer,...,g_06_a_Seria_facil,g_06_b_Seria_facil,g_06_c_Seria_facil,g_07_No,g_01_a_No_aplica,g_02_a_Dos_o_mas,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,1


Como vimos en un apartado anterior, en este modelo vamos a modificar cada variable en función de su tipo. Al conjunto de pasos que siguen los datos se le llama comúnmente **Pipelines** (literalmente, sistemas de tuberias).

In [5]:
#consumidores.info(verbose=True)
#consumidores['CatRiesgo'].value_counts()
#consumidores[consumidores['departamento'] == 'Cordoba']['CatRiesgo'].value_counts()
consumidores.shape

(10000, 40)

**INCISO**

vamos a modificar dos transformadores de scikitlearn para que sean compatibles con pipelines. Este paso es necesario en la version actual de scikit-learn, pero seguramente será arreglado en el futuro

In [6]:
class ColumnExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, columns, output_type="matrix"):
        self.columns = columns
        self.output_type = output_type

    def transform(self, X, **transform_params):
        if isinstance(X, list):
            X = pd.DataFrame.from_dict(X)
        if self.output_type == "matrix":
            return X[self.columns].to_numpy()
        elif self.output_type == "dataframe":
            return X[self.columns]
        raise Exception("output_type tiene que ser matrix o dataframe")
        
    def fit(self, X, y=None, **fit_params):
        return self

In [7]:
variable_objetivo = ["CatRiesgo"]
variables_independientes = consumidores.drop(variable_objetivo, axis=1).columns

In [8]:
datos_numericos = consumidores[variables_independientes].select_dtypes([int, float])
col_numericas = datos_numericos.columns
col_no_numericas = consumidores[variables_independientes].select_dtypes([object]).columns

dict_var_ordinales = {
    "per_edad_tipo": ["Adolescente", "Joven", "Adulto", "Persona mayor"],
    "estrato_tipo":["Estrato bajo", "Estrato medio", "Estrato alto"],
    "d_08_estado_salud_tipo":["d_08_Muy_malo","d_08_Malo","d_08_Regular","d_08_Bueno","d_08_Muy_bueno"],
    "frecuencia_consumo_marihuana_tipo":["Nunca", "Alguna vez en su vida", "Una vez en el año", "Algunas veces durante los últimos 12 meses", "Algunas veces mensualmente", "Algunas veces semanalmente", "Diariamente"],
    "frecuencia_consumo_cocaina_tipo":["Nunca", "Alguna vez en su vida", "Una vez en el año", "Algunas veces durante los últimos 12 meses", "Algunas veces mensualmente", "Algunas veces semanalmente", "Diariamente"],
    "frecuencia_consumo_basuco_tipo":["Nunca", "Alguna vez en su vida", "Una vez en el año", "Algunas veces durante los últimos 12 meses", "Algunas veces mensualmente", "Algunas veces semanalmente", "Diariamente"],
}

col_ordinales = list(dict_var_ordinales.keys())
datos_ordinales = consumidores[col_ordinales]
col_categoricas = list(set(col_no_numericas) - set(col_ordinales))
datos_categoricos = consumidores[col_categoricas]

Un pipeline de sklearn se define como una secuencia de pasos. Cada paso se define con una tupla de forma `[nombre del paso, transformador]`

Por ejemplo, si queremos crear un pipeline que procese las variables numéricas, primero imputándolas y después estandarizandolas, podriamos crear un pipeline

Creamos ahora los pipelines para cada tipo de variable. Es necesario aclarar que se va ha hacer uso de la libreria Category Encoders par ael procesamiento de variables categoricas y categoricas ordinales

In [9]:
pipeline_numerico = Pipeline([
    ["selector_numerico", ColumnExtractor(columns=col_numericas)],
    ["transformador_numerico", Pipeline([
        ("imputador_numerico", SimpleImputer(missing_values=np.nan, strategy='mean')),
        ("escalador_numerico", preprocessing.MinMaxScaler())
    ])]
])
pipeline_numerico.fit_transform(consumidores)[:5]

array([[0.30769231, 0.        ],
       [0.23076923, 0.25      ],
       [0.23076923, 0.16666667],
       [0.38461538, 0.        ],
       [0.23076923, 0.08333333]])

Ahora se procede a crear el pipeline categorico

In [10]:
pipeline_categorico = Pipeline([
    ("selector_categorica", ColumnExtractor(
        columns=col_categoricas,
        output_type="dataframe")),
    ("transformador_categorico", OneHotEncoder(
        cols=col_categoricas, handle_unknown="impute")),
])
pipeline_categorico.fit_transform(consumidores)[:5]

,d_12_b_presenta_problema_consumo_sp_barrio_tipo_1,d_12_b_presenta_problema_consumo_sp_barrio_tipo_2,g_03_curiosidad_probar_sp_tipo_1,g_03_curiosidad_probar_sp_tipo_2,g_08_b_ofrecieron_cocaina_imp_tipo_1,g_08_b_ofrecieron_cocaina_imp_tipo_2,g_08_b_ofrecieron_cocaina_imp_tipo_3,g_08_b_ofrecieron_cocaina_imp_tipo_4,g_08_b_ofrecieron_cocaina_imp_tipo_5,vivienda_tipo_1,...,departamento_25,departamento_26,departamento_27,departamento_28,departamento_29,departamento_30,departamento_31,departamento_32,g_07_alguien_ofrecio_comprar_probar_sp_tipo_1,g_07_alguien_ofrecio_comprar_probar_sp_tipo_2
0,1,0,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


`category_encoders.OrdinalEncoders` requiere un mapeado específico para cada columna, indicando el número por el que queremos reemplazar cada nivel de cada variable ordinal. Si no especificamos un mapa, asume que el orden es aleatorio.

In [11]:
niveles_ordinales = []
for col, levels in dict_var_ordinales.items():
    niveles_ordinales.append({"col": col, 
                              "mapping": dict(zip(levels, range(len(levels))))})

In [12]:
pipeline_ordinal = Pipeline([
    ("selector_ordinal", ColumnExtractor(columns=col_ordinales, output_type="dataframe")),
    ("transformador_ordinal", OrdinalEncoder(mapping=niveles_ordinales))
])
pipeline_ordinal.fit_transform(consumidores)[:5]

,per_edad_tipo,estrato_tipo,d_08_estado_salud_tipo,frecuencia_consumo_marihuana_tipo,frecuencia_consumo_cocaina_tipo,frecuencia_consumo_basuco_tipo
0,1,0,1,0,0,0
1,2,0,2,0,0,0
2,2,0,3,0,0,0
3,2,2,3,0,0,0
4,1,0,3,0,0,0


Ya tenemos una manera de, dado un conjunto de datos, separarlos y aplicar distintas transformaciones a cada variable. Nos falta una manera de, una vez se han transformado, reunirlas de nuevo.

Para ello podemos usar `FeatureUnion`, que simplemente toma un conjunto de pasos de un pipeline y los une.

In [13]:
pipeline_procesado = FeatureUnion([
    ('variables_numericas', pipeline_numerico),
    ('variables_ordinales', pipeline_ordinal),
    ('variables_categoricas', pipeline_categorico),
])

In [14]:
pipeline_procesado

FeatureUnion(transformer_list=[('variables_numericas',
                                Pipeline(steps=[('selector_numerico',
                                                 ColumnExtractor(columns=Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))),
                                                ['transformador_numerico',
                                                 Pipeline(steps=[('imputador_numerico',
                                                                  SimpleImputer()),
                                                                 ('escalador_numerico',
                                                                  MinMaxScaler())])]])),
                               ('variables_ordinales',
                                Pipeline(steps=[('select...
                                                                     'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                                     'g_02_amigos_consumen_sp_tipo',
                                                                     'g_06_c_posibilidad_conseguir_basuco_tipo',
                                                                     'g_01_a_num_familiares_consumen_sp_imp_tipo',
                                                                     'g_06_b_posibilidad_conseguir_cocaina_tipo',
                                                                     'g_04_probaria_sp_tipo',
                                                                     'g_08_c_ofrecieron_basuco_imp_tipo',
                                                                     'd_01_aporta_dinero_hogar_tipo',
                                                                     'd2_03_estado_civil_tipo',
                                                                     'departamento', ...],
                                                               handle_unknown='impute'))]))])

In [15]:
consumidores_procesado = pipeline_procesado.fit_transform(consumidores)
consumidores_procesado.shape

(10000, 145)

Finalmente, necesitamos añadir un estimador al final para predecir en base a los datos transformados, pero primero se separa los datos en datos de entrenamiento y datos de test

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
     consumidores[variables_independientes], consumidores["CatRiesgo"],
     test_size=0.20, random_state=42
)

In [17]:
estimador = RandomForestClassifier()

Se debe realizar la separacion de datos en datos de entrenamiento y datos de test y ademas se debe establecer el estimador a usar en el modelo

In [18]:
pipeline_estimador = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', estimador)
])

Para el fit se pasan los datos que se obtienen de entrenamiento y para el predict los datos de test

In [19]:
pipeline_estimador.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 Pipeline(steps=[('selector_numerico',
                                                                  ColumnExtractor(columns=Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))),
                                                                 ['transformador_numerico',
                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                   SimpleImputer()),
                                                                                  ('escalador_numerico',
                                                                                   MinMaxScaler())])]])),
                                                ('variables_ordin...
                                                                                      'g_02_amigos_consumen_sp_tipo',
                                                                                      'g_06_c_posibilidad_conseguir_basuco_tipo',
                                                                                      'g_01_a_num_familiares_consumen_sp_imp_tipo',
                                                                                      'g_06_b_posibilidad_conseguir_cocaina_tipo',
                                                                                      'g_04_probaria_sp_tipo',
                                                                                      'g_08_c_ofrecieron_basuco_imp_tipo',
                                                                                      'd_01_aporta_dinero_hogar_tipo',
                                                                                      'd2_03_estado_civil_tipo',
                                                                                      'departamento', ...],
                                                                                handle_unknown='impute'))]))])),
                ('estimador', RandomForestClassifier())])

In [20]:
predicciones = pipeline_estimador.predict(X_test)

In [21]:
predicciones

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

Se evalua el modelo segun lo obtenido

El beneficio de los pipelines, no solo es tener codigo mas legible y poder gestionar de forma ordenada todo el ciclo de vida del modelado, sino que los pipelines tienen todos los beneficios de los objetos de scikitlearn, por ejemplo, podemos usar validacion cruzada directamente con el pipeline.

In [22]:
resultados = {}

def exactitud_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.accuracy_score(y, preds)

def precision_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.precision_score(y, preds, average="weighted")

def exhaustividad_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.recall_score(y, preds, average="weighted")

def f1_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.f1_score(y, preds, average="micro")

In [23]:
def evaluar_modelo(prmPipeline,X, y, semillas):
    restultados_estimador = {}
    restultados_estimador["exactitud"] = cross_validate(prmPipeline, X, y,
                                           scoring=exactitud_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    
    restultados_estimador["precision"] = cross_validate(prmPipeline, X, y,
                                           scoring=precision_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    
    restultados_estimador["exhaustividad"] = cross_validate(prmPipeline, X, y,
                                           scoring=exhaustividad_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    
    restultados_estimador["f1_score"] = cross_validate(prmPipeline, X, y,
                                           scoring=f1_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    return restultados_estimador

In [24]:
#Funcion para poder visuallizar los resultados de las metricas
def ver_resultados():
    df_res = pd.DataFrame()
    for modelo, metricas in resultados.items():
        for metrica, valores in metricas.items():
            for medida, arreglo in valores.items():
                # Calcula la media de cada array y agrega una nueva fila al DataFrame
                df_res.loc[f"{modelo}_{metrica}", medida] = np.mean(arreglo)
                
    return df_res

In [25]:
estimador_agregado = pipeline_estimador.named_steps['estimador']
resultados["rf"] = evaluar_modelo(estimador_agregado, consumidores_procesado, consumidores.CatRiesgo, 50)

In [26]:
ver_resultados()

,fit_time,score_time,test_score,train_score
rf_exactitud,3.717787,0.028722,0.939200,1.000000
rf_precision,3.522529,0.030139,0.941923,1.000000
rf_exhaustividad,3.531219,0.030025,0.938300,1.000000
rf_f1_score,3.449244,0.028661,0.939000,0.999998


### Optimizacion de hiperparametros

In [27]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

In [28]:
param_dist_random = {
    "max_depth": [None, 10, 20, 30, 40, 50],
    "max_features": ['sqrt', 'log2'],
    "min_samples_split": sp_randint(2, 11),
    "min_samples_leaf": sp_randint(1, 11),
    "bootstrap": [True, False],
    "criterion": ["gini", "entropy"],
    "n_estimators": np.linspace(10,1000,10).astype(int)
}

In [29]:
busqueda_random = RandomizedSearchCV(estimator=estimador, 
                   param_distributions=param_dist_random,
                   scoring="accuracy", n_jobs=-1, n_iter=100)

In [30]:
pipeline_estimador_op = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random)
])

In [31]:
pipeline_estimador_op.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 Pipeline(steps=[('selector_numerico',
                                                                  ColumnExtractor(columns=Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))),
                                                                 ['transformador_numerico',
                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                   SimpleImputer()),
                                                                                  ('escalador_numerico',
                                                                                   MinMaxScaler())])]])),
                                                ('variables_ordin...
                                                         'max_depth': [None, 10,
                                                                       20, 30,
                                                                       40, 50],
                                                         'max_features': ['sqrt',
                                                                          'log2'],
                                                         'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001A805E8E1D0>,
                                                         'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001A81A799810>,
                                                         'n_estimators': array([  10,  120,  230,  340,  450,  560,  670,  780,  890, 1000])},
                                    scoring='accuracy'))])

In [32]:
pipeline_estimador_op.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [33]:
print(busqueda_random.best_score_)
print(busqueda_random.best_estimator_)

0.938375
RandomForestClassifier(bootstrap=False, max_depth=40, n_estimators=670)


In [34]:
pipeline_estimador_op_best = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random.best_estimator_)
])

In [35]:
pipeline_estimador_op_best.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 Pipeline(steps=[('selector_numerico',
                                                                  ColumnExtractor(columns=Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))),
                                                                 ['transformador_numerico',
                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                   SimpleImputer()),
                                                                                  ('escalador_numerico',
                                                                                   MinMaxScaler())])]])),
                                                ('variables_ordin...
                                                                                      'g_06_c_posibilidad_conseguir_basuco_tipo',
                                                                                      'g_01_a_num_familiares_consumen_sp_imp_tipo',
                                                                                      'g_06_b_posibilidad_conseguir_cocaina_tipo',
                                                                                      'g_04_probaria_sp_tipo',
                                                                                      'g_08_c_ofrecieron_basuco_imp_tipo',
                                                                                      'd_01_aporta_dinero_hogar_tipo',
                                                                                      'd2_03_estado_civil_tipo',
                                                                                      'departamento', ...],
                                                                                handle_unknown='impute'))]))])),
                ('estimador',
                 RandomForestClassifier(bootstrap=False, max_depth=40,
                                        n_estimators=670))])

In [36]:
pipeline_estimador_op_best.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [37]:
estimador_best = pipeline_estimador_op_best.named_steps['estimador']
resultados["rf_best"] = evaluar_modelo(estimador_best, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
rf_exactitud,3.717787,0.028722,0.939200,1.000000
rf_precision,3.522529,0.030139,0.941923,1.000000
rf_exhaustividad,3.531219,0.030025,0.938300,1.000000
rf_f1_score,3.449244,0.028661,0.939000,0.999998
rf_best_exactitud,32.110700,0.166920,0.946600,1.000000
rf_best_precision,32.102173,0.174553,0.946075,1.000000
rf_best_exhaustividad,32.150515,0.177070,0.946800,1.000000
rf_best_f1_score,33.914277,0.177469,0.945300,1.000000


### Segunda optimizacion

In [38]:
param_dist_random2 = {
    "max_depth": [None, 10, 20, 30, 40, 50],
    "max_features": ['sqrt', 'log2'],
    "min_samples_split": sp_randint(2, 15),
    "min_samples_leaf": sp_randint(1, 20),
    "bootstrap": [False],
    "criterion": ["entropy"],
    "n_estimators": sp_randint(10, 700)
}

In [39]:
busqueda_random2 = RandomizedSearchCV(estimator=estimador, 
                    param_distributions=param_dist_random2,
                   scoring="accuracy", n_jobs=-1, n_iter=100)

In [40]:
pipeline_estimador_op2 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random2)
])

In [41]:
pipeline_estimador_op2.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 Pipeline(steps=[('selector_numerico',
                                                                  ColumnExtractor(columns=Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))),
                                                                 ['transformador_numerico',
                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                   SimpleImputer()),
                                                                                  ('escalador_numerico',
                                                                                   MinMaxScaler())])]])),
                                                ('variables_ordin...
                                                         'max_features': ['sqrt',
                                                                          'log2'],
                                                         'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001A81A742A40>,
                                                         'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001A819D3FCD0>,
                                                         'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001A819D3E440>},
                                    scoring='accuracy'))])

In [42]:
pipeline_estimador_op2.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [43]:
print(busqueda_random2.best_score_)
print(busqueda_random2.best_estimator_)

0.93775
RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=20,
                       n_estimators=316)


In [44]:
pipeline_estimador_op_best2 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random2.best_estimator_)
])

In [45]:
pipeline_estimador_op_best2.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 Pipeline(steps=[('selector_numerico',
                                                                  ColumnExtractor(columns=Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))),
                                                                 ['transformador_numerico',
                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                   SimpleImputer()),
                                                                                  ('escalador_numerico',
                                                                                   MinMaxScaler())])]])),
                                                ('variables_ordin...
                                                                                      'g_01_a_num_familiares_consumen_sp_imp_tipo',
                                                                                      'g_06_b_posibilidad_conseguir_cocaina_tipo',
                                                                                      'g_04_probaria_sp_tipo',
                                                                                      'g_08_c_ofrecieron_basuco_imp_tipo',
                                                                                      'd_01_aporta_dinero_hogar_tipo',
                                                                                      'd2_03_estado_civil_tipo',
                                                                                      'departamento', ...],
                                                                                handle_unknown='impute'))]))])),
                ('estimador',
                 RandomForestClassifier(bootstrap=False, criterion='entropy',
                                        max_depth=20, n_estimators=316))])

In [46]:
pipeline_estimador_op_best2.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [47]:
estimador_best2 = pipeline_estimador_op_best2.named_steps['estimador']
resultados["rf_best_2"] = evaluar_modelo(estimador_best2, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
rf_exactitud,3.717787,0.028722,0.939200,1.000000
rf_precision,3.522529,0.030139,0.941923,1.000000
rf_exhaustividad,3.531219,0.030025,0.938300,1.000000
rf_f1_score,3.449244,0.028661,0.939000,0.999998
rf_best_exactitud,32.110700,0.166920,0.946600,1.000000
rf_best_precision,32.102173,0.174553,0.946075,1.000000
rf_best_exhaustividad,32.150515,0.177070,0.946800,1.000000
rf_best_f1_score,33.914277,0.177469,0.945300,1.000000
rf_best_2_exactitud,15.181889,0.082205,0.945400,1.000000
rf_best_2_precision,15.252404,0.081180,0.946431,1.000000


### Tercera optimizacion

In [48]:
param_dist_random3 = {
    "max_depth": [10, 20, 30, 40, 50],
    "max_features": ['sqrt', 'log2'],
    "min_samples_split": sp_randint(2, 10),
    "min_samples_leaf": sp_randint(1, 14),
    "bootstrap": [False],
    "criterion": ["entropy"],
    "n_estimators": sp_randint(10, 650)
}

In [49]:
busqueda_random3 = RandomizedSearchCV(estimator=estimador, 
                    param_distributions=param_dist_random3,
                   scoring="accuracy", n_jobs=-1, n_iter=100)

In [50]:
pipeline_estimador_op3 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random3)
])

In [51]:
pipeline_estimador_op3.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 Pipeline(steps=[('selector_numerico',
                                                                  ColumnExtractor(columns=Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))),
                                                                 ['transformador_numerico',
                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                   SimpleImputer()),
                                                                                  ('escalador_numerico',
                                                                                   MinMaxScaler())])]])),
                                                ('variables_ordin...
                                                         'max_features': ['sqrt',
                                                                          'log2'],
                                                         'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001A81A504F40>,
                                                         'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001A81A79A2F0>,
                                                         'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001A81A506410>},
                                    scoring='accuracy'))])

In [52]:
pipeline_estimador_op3.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [53]:
print(busqueda_random3.best_score_)
print(busqueda_random3.best_estimator_)

0.9395
RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=20,
                       min_samples_split=4, n_estimators=480)


In [54]:
pipeline_estimador_op_best3 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random3.best_estimator_)
])

In [55]:
pipeline_estimador_op_best3.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 Pipeline(steps=[('selector_numerico',
                                                                  ColumnExtractor(columns=Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))),
                                                                 ['transformador_numerico',
                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                   SimpleImputer()),
                                                                                  ('escalador_numerico',
                                                                                   MinMaxScaler())])]])),
                                                ('variables_ordin...
                                                                                      'g_01_a_num_familiares_consumen_sp_imp_tipo',
                                                                                      'g_06_b_posibilidad_conseguir_cocaina_tipo',
                                                                                      'g_04_probaria_sp_tipo',
                                                                                      'g_08_c_ofrecieron_basuco_imp_tipo',
                                                                                      'd_01_aporta_dinero_hogar_tipo',
                                                                                      'd2_03_estado_civil_tipo',
                                                                                      'departamento', ...],
                                                                                handle_unknown='impute'))]))])),
                ('estimador',
                 RandomForestClassifier(bootstrap=False, criterion='entropy',
                                        max_depth=20, min_samples_split=4,
                                        n_estimators=480))])

In [56]:
pipeline_estimador_op_best3.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [57]:
estimador_best3 = pipeline_estimador_op_best3.named_steps['estimador']
resultados["rf_best_3"] = evaluar_modelo(estimador_best3, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
rf_exactitud,3.717787,0.028722,0.939200,1.000000
rf_precision,3.522529,0.030139,0.941923,1.000000
rf_exhaustividad,3.531219,0.030025,0.938300,1.000000
rf_f1_score,3.449244,0.028661,0.939000,0.999998
rf_best_exactitud,32.110700,0.166920,0.946600,1.000000
rf_best_precision,32.102173,0.174553,0.946075,1.000000
rf_best_exhaustividad,32.150515,0.177070,0.946800,1.000000
rf_best_f1_score,33.914277,0.177469,0.945300,1.000000
rf_best_2_exactitud,15.181889,0.082205,0.945400,1.000000
rf_best_2_precision,15.252404,0.081180,0.946431,1.000000


### Cuarta optimizacion

In [58]:
param_dist_random4 = {
    "max_depth": [20, 30, 40, 50],
    "max_features": ['sqrt', 'log2'],
    "min_samples_split": sp_randint(2, 6),
    "min_samples_leaf": sp_randint(1, 6),
    "bootstrap": [False],
    "criterion": ["entropy"],
    "n_estimators": sp_randint(10, 650)
}

In [59]:
busqueda_random4 = RandomizedSearchCV(estimator=estimador, 
                    param_distributions=param_dist_random4,
                   scoring="accuracy", n_jobs=-1, n_iter=100)

In [60]:
pipeline_estimador_op4 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random4)
])

In [61]:
pipeline_estimador_op4.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 Pipeline(steps=[('selector_numerico',
                                                                  ColumnExtractor(columns=Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))),
                                                                 ['transformador_numerico',
                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                   SimpleImputer()),
                                                                                  ('escalador_numerico',
                                                                                   MinMaxScaler())])]])),
                                                ('variables_ordin...
                                                         'max_features': ['sqrt',
                                                                          'log2'],
                                                         'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001A81A25A2F0>,
                                                         'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001A81A7B0400>,
                                                         'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001A81A25B430>},
                                    scoring='accuracy'))])

In [62]:
pipeline_estimador_op4.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [63]:
print(busqueda_random4.best_score_)
print(busqueda_random4.best_estimator_)

0.93925
RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=30,
                       min_samples_split=5, n_estimators=632)


In [64]:
pipeline_estimador_op_best4 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random4.best_estimator_)
])

In [65]:
pipeline_estimador_op_best4.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 Pipeline(steps=[('selector_numerico',
                                                                  ColumnExtractor(columns=Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))),
                                                                 ['transformador_numerico',
                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                   SimpleImputer()),
                                                                                  ('escalador_numerico',
                                                                                   MinMaxScaler())])]])),
                                                ('variables_ordin...
                                                                                      'g_01_a_num_familiares_consumen_sp_imp_tipo',
                                                                                      'g_06_b_posibilidad_conseguir_cocaina_tipo',
                                                                                      'g_04_probaria_sp_tipo',
                                                                                      'g_08_c_ofrecieron_basuco_imp_tipo',
                                                                                      'd_01_aporta_dinero_hogar_tipo',
                                                                                      'd2_03_estado_civil_tipo',
                                                                                      'departamento', ...],
                                                                                handle_unknown='impute'))]))])),
                ('estimador',
                 RandomForestClassifier(bootstrap=False, criterion='entropy',
                                        max_depth=30, min_samples_split=5,
                                        n_estimators=632))])

In [66]:
pipeline_estimador_op_best4.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [67]:
estimador_best4 = pipeline_estimador_op_best4.named_steps['estimador']
resultados["rf_best_4"] = evaluar_modelo(estimador_best4, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
rf_exactitud,3.717787,0.028722,0.939200,1.000000
rf_precision,3.522529,0.030139,0.941923,1.000000
rf_exhaustividad,3.531219,0.030025,0.938300,1.000000
rf_f1_score,3.449244,0.028661,0.939000,0.999998
rf_best_exactitud,32.110700,0.166920,0.946600,1.000000
rf_best_precision,32.102173,0.174553,0.946075,1.000000
rf_best_exhaustividad,32.150515,0.177070,0.946800,1.000000
rf_best_f1_score,33.914277,0.177469,0.945300,1.000000
rf_best_2_exactitud,15.181889,0.082205,0.945400,1.000000
rf_best_2_precision,15.252404,0.081180,0.946431,1.000000


### Quinta optimizacion

In [68]:
param_dist_random5 = {
    "max_depth": [20, 30, 40, 50, 60],
    "max_features": ['sqrt', 'log2'],
    "min_samples_split": sp_randint(2, 6),
    "min_samples_leaf": sp_randint(1, 6),
    "bootstrap": [False],
    "criterion": ["entropy"],
    "n_estimators": sp_randint(10, 650)
}

In [69]:
busqueda_random5 = RandomizedSearchCV(estimator=estimador, 
                    param_distributions=param_dist_random5,
                   scoring="accuracy", n_jobs=-1, n_iter=100)

In [70]:
pipeline_estimador_op5 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random5)
])

In [71]:
pipeline_estimador_op5.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 Pipeline(steps=[('selector_numerico',
                                                                  ColumnExtractor(columns=Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))),
                                                                 ['transformador_numerico',
                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                   SimpleImputer()),
                                                                                  ('escalador_numerico',
                                                                                   MinMaxScaler())])]])),
                                                ('variables_ordin...
                                                         'max_features': ['sqrt',
                                                                          'log2'],
                                                         'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001A819D3EDA0>,
                                                         'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001A81A25B580>,
                                                         'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001A81A26CFD0>},
                                    scoring='accuracy'))])

In [72]:
pipeline_estimador_op5.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [73]:
print(busqueda_random5.best_score_)
print(busqueda_random5.best_estimator_)

0.939125
RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=40,
                       min_samples_split=3, n_estimators=442)


In [74]:
pipeline_estimador_op_best5 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random5.best_estimator_)
])

In [75]:
pipeline_estimador_op_best5.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 Pipeline(steps=[('selector_numerico',
                                                                  ColumnExtractor(columns=Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))),
                                                                 ['transformador_numerico',
                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                   SimpleImputer()),
                                                                                  ('escalador_numerico',
                                                                                   MinMaxScaler())])]])),
                                                ('variables_ordin...
                                                                                      'g_01_a_num_familiares_consumen_sp_imp_tipo',
                                                                                      'g_06_b_posibilidad_conseguir_cocaina_tipo',
                                                                                      'g_04_probaria_sp_tipo',
                                                                                      'g_08_c_ofrecieron_basuco_imp_tipo',
                                                                                      'd_01_aporta_dinero_hogar_tipo',
                                                                                      'd2_03_estado_civil_tipo',
                                                                                      'departamento', ...],
                                                                                handle_unknown='impute'))]))])),
                ('estimador',
                 RandomForestClassifier(bootstrap=False, criterion='entropy',
                                        max_depth=40, min_samples_split=3,
                                        n_estimators=442))])

In [76]:
pipeline_estimador_op_best5.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [77]:
estimador_best5 = pipeline_estimador_op_best5.named_steps['estimador']
resultados["rf_best_5"] = evaluar_modelo(estimador_best5, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
rf_exactitud,3.717787,0.028722,0.939200,1.000000
rf_precision,3.522529,0.030139,0.941923,1.000000
rf_exhaustividad,3.531219,0.030025,0.938300,1.000000
rf_f1_score,3.449244,0.028661,0.939000,0.999998
rf_best_exactitud,32.110700,0.166920,0.946600,1.000000
rf_best_precision,32.102173,0.174553,0.946075,1.000000
rf_best_exhaustividad,32.150515,0.177070,0.946800,1.000000
rf_best_f1_score,33.914277,0.177469,0.945300,1.000000
rf_best_2_exactitud,15.181889,0.082205,0.945400,1.000000
rf_best_2_precision,15.252404,0.081180,0.946431,1.000000


### Exportar con joblib el mejor pipeline obtenido

Tenemos que exportar 3 cosas para poder recrear el Pipeline:

**1. El pipeline entrenado.**

Podemos usar la libreria de scikit-learn `joblib` para guardar nuestro pipeline al disco duro. Joblib es una libreria basada en pickle que sirve para guardar objetos de python al disco duro, pero que está optimizado para scikit-learn.

In [78]:
import joblib

In [79]:
joblib.dump(pipeline_estimador_op_best5, 'pipeline_rf_best.pkl')

['pipeline_rf_best.pkl']

In [ ]:
clf = joblib.load('C:/Users/Danny\Desktop\Analisis_Modelo\Alisis Danny\4 - Pipelines\Modelo RF/pipeline_rf_best.pkl')

In [ ]:
clf.predict(consumidores.head())

**2. Los nombres de las columnas**

Necesitamos saber las columnas que espera el pipeline para poder saber cómo tenemos que formatear los datos en el futuro de forma que los reconozca.

In [ ]:
import json
with open('columnas_consumidores.json', 'w') as fname:
    consumidores_columnas = consumidores.columns.tolist()
    json.dump(consumidores_columnas, fname)

**3. Los tipos de datos de cada variable**

Necesitamos que las variables de las nuevas observaciones estén en el mismo formato que espera el pipeline.

In [ ]:
consumidores_dtypes = consumidores.dtypes
consumidores_dtypes = {col: consumidores[col].dtype for col in consumidores.columns}
joblib.dump(consumidores_dtypes, 'dtypes_consumidores.pkl')